## Load data into Spark

This assignment was accomplished using IBM's Data Science Expereience, from which a 5 node Spark cluster was provisioned. There is 30 day free trial to use IBM DSE.

The data was loaded using:

```python
sc.textFile("swift://SparkTest." + name + "/2015_07_22_mktplace_shop_web_log_sample.log")
```

To peek into the datset we use:

```python
log_rdd.take(2)
```


In [1]:

# This function accesses a file in your Object Storage. 

def set_hadoop_config_with_credentials_f54697639170483e9617aff099ae997b(name):
    """This function sets the Hadoop configuration so it is possible to
    access data from Bluemix Object Storage V3 using Spark"""

    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', '648a7a8047c546109acb2f577f7634ff')
    hconf.set(prefix + '.username', 'bd3c798d89cc49599610f0f675c9e3dc')
    hconf.set(prefix + '.password', 'P[5syRKMGcm.h437')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', False)

name = 'keystone'
set_hadoop_config_with_credentials_f54697639170483e9617aff099ae997b(name)

log_rdd = sc.textFile("swift://SparkTest." + name + "/2015_07_22_mktplace_shop_web_log_sample.log")

# Printing out to see what the data looks 

log_rdd.take(2)

[u'2015-07-22T09:00:28.019143Z marketpalce-shop 123.242.248.130:54635 10.0.6.158:80 0.000022 0.026109 0.00002 200 200 0 699 "GET https://paytm.com:443/shop/authresponse?code=f2405b05-e2ee-4b0d-8f6a-9fed0fcfe2e0&state=null HTTP/1.1" "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.130 Safari/537.36" ECDHE-RSA-AES128-GCM-SHA256 TLSv1.2',
 u'2015-07-22T09:00:27.894580Z marketpalce-shop 203.91.211.44:51402 10.0.4.150:80 0.000024 0.15334 0.000026 200 200 0 1497 "GET https://paytm.com:443/shop/wallet/txnhistory?page_size=10&page_number=0&channel=web&version=2 HTTP/1.1" "Mozilla/5.0 (Windows NT 6.1; rv:39.0) Gecko/20100101 Firefox/39.0" ECDHE-RSA-AES128-GCM-SHA256 TLSv1.2']

## Parsing & Sorting

First step was to parse the log data and for the best step, I initially started writing regular expressions, but then noticed in the Readme.md that the the logs were taken from an Amazon ELB. I have worked with Logstash before and was exposed to ** grok filters ** which are basically templated regular expresisons, and there is prebuilt grokfilter for ELB: *'%{ELB_ACCESS_LOG}' *

In [2]:
! pip install pygrok

In [3]:
from pygrok import Grok
pattern = '%{ELB_ACCESS_LOG}'
grok = Grok(pattern)
parsed_rdd = log_rdd.map(lambda line : grok.match(line))
sorted_rdd = parsed_rdd.sortBy(lambda d: d.get('timestamp')).cache()

Once parsed, the resultant rdd is sorted according to timestamp in order to be used for sessionization.
The sorting is done using a lambda expression where we use ".get()" since every element in the rdd is a dict and we know that particular element we want it sorted against.

We also **cache** the sorted_rdd, as we will be using this again.

In [4]:
parsed_rdd.take(3)

[{'backend_processing_time': 0.026109,
  'backend_response': 200,
  'backendip': u'10.0.6.158',
  'backendport': 80,
  'bytes': 699,
  'clientip': u'123.242.248.130',
  'clientport': 54635,
  'elb': u'marketpalce-shop',
  'httpversion': u'1.1',
  'params': u'?code=f2405b05-e2ee-4b0d-8f6a-9fed0fcfe2e0&state=null',
  'path': u'/shop/authresponse',
  'port': u'443',
  'proto': u'https',
  'rawrequest': None,
  'received_bytes': 0,
  'request': u'https://paytm.com:443/shop/authresponse?code=f2405b05-e2ee-4b0d-8f6a-9fed0fcfe2e0&state=null',
  'request_processing_time': 2.2e-05,
  'response': 200,
  'response_processing_time': 2e-05,
  'timestamp': u'2015-07-22T09:00:28.019143Z',
  'urihost': u'paytm.com:443',
  'verb': u'GET'},
 {'backend_processing_time': 0.15334,
  'backend_response': 200,
  'backendip': u'10.0.4.150',
  'backendport': 80,
  'bytes': 1497,
  'clientip': u'203.91.211.44',
  'clientport': 51402,
  'elb': u'marketpalce-shop',
  'httpversion': u'1.1',
  'params': u'?page_size

In [5]:
sorted_rdd.take(3)

[{'backend_processing_time': 0.274619,
  'backend_response': 200,
  'backendip': u'10.0.6.158',
  'backendport': 80,
  'bytes': 872,
  'clientip': u'106.51.235.133',
  'clientport': 1813,
  'elb': u'marketpalce-shop',
  'httpversion': u'1.1',
  'params': u'?page_size=10&page_number=0&channel=web&version=2',
  'path': u'/shop/wallet/txnhistory',
  'port': u'443',
  'proto': u'https',
  'rawrequest': None,
  'received_bytes': 0,
  'request': u'https://paytm.com:443/shop/wallet/txnhistory?page_size=10&page_number=0&channel=web&version=2',
  'request_processing_time': 2e-05,
  'response': 200,
  'response_processing_time': 2.2e-05,
  'timestamp': u'2015-07-22T02:40:06.499174Z',
  'urihost': u'paytm.com:443',
  'verb': u'GET'},
 {'backend_processing_time': 0.258738,
  'backend_response': 200,
  'backendip': u'10.0.4.227',
  'backendport': 80,
  'bytes': 212,
  'clientip': u'115.250.16.146',
  'clientport': 63523,
  'elb': u'marketpalce-shop',
  'httpversion': u'1.1',
  'params': u'?channel=

We print the above results of parsed_rdd and sorted_rdd and check if the results are sorted.

## Conversion to Pandas

### Problems

1. Since the parsed rdd were basically a list of dict objects, converting it into pandas was not straight forward by using

```python
rdd.toDF().toPandas()
```
### Solution

After refering [Fixing PySpark Schemas](http://nadbordrozd.github.io/blog/2016/05/22/one-weird-trick-that-will-fix-your-pyspark-schemas/)

I was able to define a prototype by using one of the example of sorted_rdd above instead of defining struct types as below:

```python
schema = StructType([
        StructField("first_name", StringType(), True),
        StructField("last_name", StringType(), True),
        StructField("occupation", StringType(), True)])
```

and then create a dataframe out of an rdd of dicts, with schema inferred from a prototype record.

In [6]:
prototype = {'backend_processing_time': 0.026109,
  'backend_response': 200,
  'backendip': u'10.0.6.158',
  'backendport': 80,
  'bytes': 699,
  'clientip': u'123.242.248.130',
  'clientport': 54635,
  'elb': u'marketpalce-shop',
  'httpversion': u'1.1',
  'params': u'?code=f2405b05-e2ee-4b0d-8f6a-9fed0fcfe2e0&state=null',
  'path': u'/shop/authresponse',
  'port': u'443',
  'proto': u'https',
  'rawrequest': u'/shop/authresponse',
  'received_bytes': 0,
  'request': u'https://paytm.com:443/shop/authresponse?code=f2405b05-e2ee-4b0d-8f6a-9fed0fcfe2e0&state=null',
  'request_processing_time': 2.2e-05,
  'response': 200,
  'response_processing_time': 2e-05,
  'timestamp': u'2015-07-22T09:00:28.019143Z',
  'urihost': u'paytm.com:443',
  'verb': u'GET'}

In [7]:
import pyspark.sql.types as pst
from pyspark.sql import Row

def infer_schema(rec):
    """infers dataframe schema for a record. Assumes every dict is a Struct, not a Map"""
    if isinstance(rec, dict):
        return pst.StructType([pst.StructField(key, infer_schema(value), True)
                              for key, value in sorted(rec.items())])
    elif isinstance(rec, list):
        if len(rec) == 0:
            raise ValueError("can't infer type of an empty list")
        elem_type = infer_schema(rec[0])
        for elem in rec:
            this_type = infer_schema(elem)
            if elem_type != this_type:
                raise ValueError("can't infer type of a list with inconsistent elem types")
        return pst.ArrayType(elem_type)
    else:
        return pst._infer_type(rec)

In [8]:
import pyspark.sql.types as pst
from pyspark.sql import Row

def infer_schema(rec):
    """infers dataframe schema for a record. Assumes every dict is a Struct, not a Map"""
    if isinstance(rec, dict):
        return pst.StructType([pst.StructField(key, infer_schema(value), True)
                              for key, value in sorted(rec.items())])
    elif isinstance(rec, list):
        if len(rec) == 0:
            raise ValueError("can't infer type of an empty list")
        elem_type = infer_schema(rec[0])
        for elem in rec:
            this_type = infer_schema(elem)
            if elem_type != this_type:
                raise ValueError("can't infer type of a list with inconsistent elem types")
        return pst.ArrayType(elem_type)
    else:
        return pst._infer_type(rec)

In [9]:
def _rowify(x, prototype):
    """creates a Row object conforming to a schema as specified by a dict"""

    def _equivalent_types(x, y):
        if type(x) in [str, unicode] and type(y) in [str, unicode]:
            return True
        return isinstance(x, type(y)) or isinstance(y, type(x))

    if x is None:
        return None
    elif isinstance(prototype, dict):
        if type(x) != dict:
            raise ValueError("expected dict, got %s instead" % type(x))
        rowified_dict = {}
        for key, val in x.items():
            if key not in prototype:
                raise ValueError("got unexpected field %s" % key)
            rowified_dict[key] = _rowify(val, prototype[key])
            for key in prototype:
                if key not in x:
                    raise ValueError(
                        "expected %s field but didn't find it" % key)
        return Row(**rowified_dict)
    elif isinstance(prototype, list):
        if type(x) != list:
            raise ValueError("expected list, got %s instead" % type(x))
        return [_rowify(e, prototype[0]) for e in x]
    else:
        if not _equivalent_types(x, prototype):
            raise ValueError("expected %s, got %s instead" %
                             (type(prototype), type(x)))
        return x

In [10]:
def df_from_rdd(rdd, prototype, sql):
    """creates a dataframe out of an rdd of dicts, with schema inferred from a prototype record"""
    schema = infer_schema(prototype)
    row_rdd = rdd.map(lambda x: _rowify(x, prototype))
    return sql.createDataFrame(row_rdd, schema)

## Pandas

As you can see below, we have converted the rdd of dict objects to pandas, where we can carry out our analysis further.

In [11]:
pdf = df_from_rdd(sorted_rdd,prototype,sqlContext).toPandas()

In [12]:
pdf.tail(10)

,backend_processing_time,backend_response,backendip,backendport,bytes,clientip,clientport,elb,httpversion,params,...,proto,rawrequest,received_bytes,request,request_processing_time,response,response_processing_time,timestamp,urihost,verb
1158490,0.007372,200,10.0.4.176,80,1125,106.186.23.95,43944,marketpalce-shop,1.0,?parent_id=1782097&price=499,...,https,None,0,https://paytm.com:443/papi/v1/promosearch/prod...,0.000019,200,0.000020,2015-07-22T21:10:27.886244Z,paytm.com:443,GET
1158491,0.030922,200,10.0.6.178,80,385,207.244.77.21,65411,marketpalce-shop,1.1,?parent_id=6331562&price=17400&channel=web&ver...,...,https,None,0,https://paytm.com:443/papi/v1/promosearch/prod...,0.000024,200,0.000020,2015-07-22T21:10:27.891764Z,paytm.com:443,GET
1158492,0.005931,200,10.0.6.199,80,2,207.244.77.21,42468,marketpalce-shop,1.1,?merchant_id=44532&channel=web&version=2,...,https,None,0,https://paytm.com:443/papi/nps/merchantrating?...,0.000024,200,0.000021,2015-07-22T21:10:27.899144Z,paytm.com:443,GET
1158493,0.003958,200,10.0.4.225,80,13820,112.196.25.164,42792,marketpalce-shop,1.1,None,...,https,None,0,https://paytm.com:443/bus-tickets/search/Delhi...,0.000024,200,0.000021,2015-07-22T21:10:27.914275Z,paytm.com:443,GET
1158494,0.009750,401,10.0.4.227,80,95,54.169.191.85,15462,marketpalce-shop,1.1,None,...,https,None,0,https://paytm.com:443/recharges/547cafb3e4b04b...,0.000024,401,0.000022,2015-07-22T21:10:27.933107Z,paytm.com:443,GET
1158495,0.009275,401,10.0.6.158,80,95,54.169.191.85,15462,marketpalce-shop,1.1,None,...,https,None,0,https://paytm.com:443/recharges/547cafb3e4b04b...,0.000029,401,0.000021,2015-07-22T21:10:27.945346Z,paytm.com:443,GET
1158496,0.003958,200,10.0.4.244,80,585,52.74.219.71,39790,marketpalce-shop,1.0,?parent_id=0&price=635,...,https,None,0,https://paytm.com:443/papi/v1/promosearch/prod...,0.000017,200,0.000020,2015-07-22T21:10:27.950160Z,paytm.com:443,GET
1158497,0.012966,200,10.0.4.176,80,598,106.186.23.95,43947,marketpalce-shop,1.0,?parent_id=10338784&price=310,...,https,None,0,https://paytm.com:443/papi/v1/promosearch/prod...,0.000017,200,0.000022,2015-07-22T21:10:27.952944Z,paytm.com:443,GET
1158498,0.000269,200,10.0.4.217,80,1150,122.160.79.97,4237,marketpalce-shop,1.1,None,...,https,None,0,https://paytm.com:443/favicon.ico,0.000023,200,0.000021,2015-07-22T21:10:27.977284Z,paytm.com:443,GET
1158499,0.006732,200,10.0.4.227,80,51,207.244.77.21,38674,marketpalce-shop,1.1,?channel=web&version=2,...,https,None,0,https://paytm.com:443/papi/rr/products/1666638...,0.000023,200,0.000020,2015-07-22T21:10:27.993803Z,paytm.com:443,GET


In [13]:
pdf.shape

(1158500, 22)

In [14]:
pdf.columns.values

array(['backend_processing_time', 'backend_response', 'backendip',
       'backendport', 'bytes', 'clientip', 'clientport', 'elb',
       'httpversion', 'params', 'path', 'port', 'proto', 'rawrequest',
       'received_bytes', 'request', 'request_processing_time', 'response',
       'response_processing_time', 'timestamp', 'urihost', 'verb'], dtype=object)

## Filtering and organizing the columns

For processing we only need the below columns:

```python
['timestamp','clientip','elb','verb','request_processing_time','response_processing_time','backend_processing_time','urihost','request']
```

In [15]:
finaldf = pdf[['timestamp','clientip','elb','verb','request_processing_time','response_processing_time',
              'backend_processing_time','urihost','request']]

Making sure of the number of rows

In [16]:
len(finaldf)

1158500

In [17]:
finaldf.head(2)

,timestamp,clientip,elb,verb,request_processing_time,response_processing_time,backend_processing_time,urihost,request
0,2015-07-22T02:40:06.499174Z,106.51.235.133,marketpalce-shop,GET,0.000020,0.000022,0.274619,paytm.com:443,https://paytm.com:443/shop/wallet/txnhistory?p...
1,2015-07-22T02:40:06.632589Z,115.250.16.146,marketpalce-shop,GET,0.000023,0.000021,0.258738,paytm.com:443,https://paytm.com:443/shop/wallet/balance?chan...


## Convert timestamp to pandas timestamp

In [18]:
import pandas as pd
finaldf.timestamp = pd.to_datetime(finaldf.timestamp)

/usr/local/src/bluemix_jupyter_bundle.v31/notebook/lib/python2.7/site-packages/pandas/core/generic.py:2387: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [19]:
finaldf.head(3)

,timestamp,clientip,elb,verb,request_processing_time,response_processing_time,backend_processing_time,urihost,request
0,2015-07-22 02:40:06.499174,106.51.235.133,marketpalce-shop,GET,0.000020,0.000022,0.274619,paytm.com:443,https://paytm.com:443/shop/wallet/txnhistory?p...
1,2015-07-22 02:40:06.632589,115.250.16.146,marketpalce-shop,GET,0.000023,0.000021,0.258738,paytm.com:443,https://paytm.com:443/shop/wallet/balance?chan...
2,2015-07-22 02:40:06.633226,106.51.235.133,marketpalce-shop,GET,0.000021,0.000020,0.066167,paytm.com:443,https://paytm.com:443/shop/wallet/balance?chan...


In [20]:
finaldf.shape

(1158500, 9)

## Question 1:  Sessionization

Here we determine a "new_session" which is inferred by comparing current timestamp to previous timestamp and checking if the difference between the two exceeds the threshold value (15 min)

1. We add a column called *** prev_mytimestamp ***

In [21]:
sessiondf = finaldf[['clientip','timestamp']]

sessiondf =  pd.concat([sessiondf,
                       sessiondf.groupby('clientip').transform(lambda x:x.shift(1))]
                      ,axis=1)
sessiondf.columns = ['clientip','timestamp','prev_mytimestamp']

2. We create a ***time_diff*** which is the difference betwwen the current and the previous timestamp.
   
3. Based on the resultant ***time_diff***, we can create a new session which is inferred after 15 minutes of inactivity or whaterevr the set threshold    value is and the outcome is ***new_session*** column where a value of 1 indicates a new session.


In [22]:
from datetime import timedelta

threshold = timedelta(seconds=15*60)
sessiondf['time_diff'] = sessiondf['timestamp'] - sessiondf['prev_mytimestamp']
sessiondf['new_session'] = ((sessiondf['time_diff'])>=threshold).astype(int)

In [23]:
sessiondf.sample(10)

,clientip,timestamp,prev_mytimestamp,time_diff,new_session
69079,54.169.20.106,2015-07-22 05:13:35.247113,2015-07-22 05:13:35.228474,00:00:00.018639,0
934880,122.177.94.119,2015-07-22 16:43:54.545945,2015-07-22 16:43:54.068892,00:00:00.477053,0
316022,167.184.214.6,2015-07-22 10:33:20.315459,2015-07-22 10:33:20.223605,00:00:00.091854,0
1003031,101.212.66.127,2015-07-22 17:42:49.619277,2015-07-22 17:42:02.562296,00:00:47.056981,0
576521,182.69.48.186,2015-07-22 11:01:23.739217,2015-07-22 11:01:23.535170,00:00:00.204047,0
222385,49.14.141.17,2015-07-22 09:03:25.761298,2015-07-22 09:03:24.613476,00:00:01.147822,0
835677,117.212.228.156,2015-07-22 16:24:12.382081,NaT,NaT,0
249669,37.228.105.38,2015-07-22 09:04:33.162332,2015-07-22 09:04:33.159245,00:00:00.003087,0
548008,52.74.219.71,2015-07-22 10:49:36.361058,2015-07-22 10:49:36.346148,00:00:00.014910,0
356002,117.214.226.170,2015-07-22 10:35:34.056395,NaT,NaT,0


##### Perform a sanity check by checking against a particular URL, below you should see a new session defined when the time_diff exceeds the threshold value

In [24]:
sessiondf[sessiondf['clientip'] == '173.252.120.112']

,clientip,timestamp,prev_mytimestamp,time_diff,new_session
229253,173.252.120.112,2015-07-22 09:03:44.079435,NaT,NaT,0
229728,173.252.120.112,2015-07-22 09:03:45.322027,2015-07-22 09:03:44.079435,00:00:01.242592,0
371698,173.252.120.112,2015-07-22 10:36:16.599327,2015-07-22 09:03:45.322027,01:32:31.277300,1
436879,173.252.120.112,2015-07-22 10:39:03.109423,2015-07-22 10:36:16.599327,00:02:46.510096,0
484377,173.252.120.112,2015-07-22 10:46:47.816462,2015-07-22 10:39:03.109423,00:07:44.707039,0
762853,173.252.120.112,2015-07-22 16:20:56.714741,2015-07-22 10:46:47.816462,05:34:08.898279,1
936710,173.252.120.112,2015-07-22 16:43:58.854323,2015-07-22 16:20:56.714741,00:23:02.139582,1
965110,173.252.120.112,2015-07-22 17:41:04.356608,2015-07-22 16:43:58.854323,00:57:05.502285,1
988127,173.252.120.112,2015-07-22 17:42:11.699510,2015-07-22 17:41:04.356608,00:01:07.342902,0
1098330,173.252.120.112,2015-07-22 18:03:32.708049,2015-07-22 17:42:11.699510,00:21:21.008539,1


### Creating session_id

We perform a cumulative sum over the new_session column and  concatenate it with the clientip which forms the basis of the session_id column

In [25]:
sessiondf['session_id'] = sessiondf['clientip'].astype(str) + '_' + sessiondf.groupby("clientip")['new_session'].cumsum().astype(str)

In [26]:
sessiondf.sample(10)

,clientip,timestamp,prev_mytimestamp,time_diff,new_session,session_id
1145386,182.66.52.146,2015-07-22 21:07:51.825132,2015-07-22 21:07:42.344746,00:00:09.480386,0,182.66.52.146_0
63707,14.99.168.76,2015-07-22 05:13:13.632380,2015-07-22 05:13:13.632115,00:00:00.000265,0,14.99.168.76_0
67934,203.99.196.126,2015-07-22 05:13:30.168275,2015-07-22 05:13:30.139202,00:00:00.029073,0,203.99.196.126_0
586708,117.233.136.202,2015-07-22 11:01:49.388235,2015-07-22 11:01:49.120065,00:00:00.268170,0,117.233.136.202_0
475728,182.68.238.255,2015-07-22 10:46:26.318923,2015-07-22 10:46:16.171222,00:00:10.147701,0,182.68.238.255_0
425023,182.71.68.2,2015-07-22 10:38:31.938650,2015-07-22 10:38:31.251862,00:00:00.686788,0,182.71.68.2_0
548632,115.111.64.106,2015-07-22 10:49:38.181245,2015-07-22 10:49:33.961239,00:00:04.220006,0,115.111.64.106_1
724712,171.60.101.215,2015-07-22 16:13:32.818420,2015-07-22 16:13:26.828313,00:00:05.990107,0,171.60.101.215_0
1002374,101.212.67.141,2015-07-22 17:42:48.856160,2015-07-22 17:42:44.202398,00:00:04.653762,0,101.212.67.141_0
677842,54.169.77.128,2015-07-22 16:11:31.512541,2015-07-22 16:11:31.499094,00:00:00.013447,0,54.169.77.128_2


In [27]:
sessiondf[sessiondf['clientip'] == '173.252.120.112']

,clientip,timestamp,prev_mytimestamp,time_diff,new_session,session_id
229253,173.252.120.112,2015-07-22 09:03:44.079435,NaT,NaT,0,173.252.120.112_0
229728,173.252.120.112,2015-07-22 09:03:45.322027,2015-07-22 09:03:44.079435,00:00:01.242592,0,173.252.120.112_0
371698,173.252.120.112,2015-07-22 10:36:16.599327,2015-07-22 09:03:45.322027,01:32:31.277300,1,173.252.120.112_1
436879,173.252.120.112,2015-07-22 10:39:03.109423,2015-07-22 10:36:16.599327,00:02:46.510096,0,173.252.120.112_1
484377,173.252.120.112,2015-07-22 10:46:47.816462,2015-07-22 10:39:03.109423,00:07:44.707039,0,173.252.120.112_1
762853,173.252.120.112,2015-07-22 16:20:56.714741,2015-07-22 10:46:47.816462,05:34:08.898279,1,173.252.120.112_2
936710,173.252.120.112,2015-07-22 16:43:58.854323,2015-07-22 16:20:56.714741,00:23:02.139582,1,173.252.120.112_3
965110,173.252.120.112,2015-07-22 17:41:04.356608,2015-07-22 16:43:58.854323,00:57:05.502285,1,173.252.120.112_4
988127,173.252.120.112,2015-07-22 17:42:11.699510,2015-07-22 17:41:04.356608,00:01:07.342902,0,173.252.120.112_4
1098330,173.252.120.112,2015-07-22 18:03:32.708049,2015-07-22 17:42:11.699510,00:21:21.008539,1,173.252.120.112_5


## Question 2: Find the average session time

In [28]:
grouped_df = sessiondf.groupby('session_id')

In [29]:
avg_df = grouped_df['time_diff'].sum() / grouped_df['time_diff'].count()

In [30]:
average_df = avg_df.to_frame('average_session_time').reset_index(True)

In [31]:
average_df.head()

,session_id,average_session_time
0,1.186.101.79_0,00:00:34.923210
1,1.186.103.240_0,00:00:01.935169
2,1.186.103.78_0,00:00:03.000161
3,1.186.108.213_0,00:01:52.907065
4,1.186.108.230_0,00:00:04.703801


## Question 3: Determine unique URL visits per session. To clarify, count a hit to a unique URL only once per session.

In [32]:
session_result = pd.DataFrame({'unique_url_per_session' : sessiondf.groupby('clientip')['new_session'].sum() + 1}).reset_index()

In [33]:
session_result.sort('unique_url_per_session', ascending=False)

/usr/local/src/bluemix_jupyter_bundle.v31/notebook/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,clientip,unique_url_per_session
72042,220.226.206.7,13
84064,54.183.255.140,10
42244,119.81.61.166,10
84179,54.251.31.140,10
84175,54.250.253.236,10
84174,54.248.220.44,10
84173,54.245.168.44,10
84172,54.244.52.204,10
84171,54.243.31.236,10
84170,54.241.32.108,10


## Question 4 : Find the most engaged users, ie the IPs with the longest session times

We reuse the results of the average session time to sort and display the first 10 engaged users

In [34]:
most_engaged = average_df.sort('average_session_time',ascending=False)

/usr/local/src/bluemix_jupyter_bundle.v31/notebook/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [35]:
most_engaged.nlargest(10,'average_session_time').reset_index(True)

,index,session_id,average_session_time
0,92199,27.120.106.3_1,18:24:58.917758
1,49250,117.255.253.155_1,15:57:02.780406
2,10998,103.24.125.26_1,15:54:46.056403
3,69328,150.228.40.140_1,15:53:33.795814
4,108437,66.249.82.186_1,15:52:07.160237
5,70917,168.235.194.47_1,15:24:35.792257
6,24151,107.167.109.55_1,15:21:53.394211
7,103409,59.177.1.75_1,15:19:11.434379
8,2227,1.38.21.92_1,15:02:48.503090
9,11923,103.25.0.50_1,15:02:40.998829
